<a href="https://colab.research.google.com/github/james-monahan/Code-school-notebooks/blob/main/Week-8-pandas-sql/Yuka_Part1_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# The dataset

In this challenge we will use the following dataset: [dataset food](https://drive.google.com/open?id=1L4tDDnZWwPDjeRKlleoYHgic2_xNhZ8i). A little warning : it is huge!

You will use it to conduct a descriptive and exploratory analysis.
The goal is achieved when you end up having a dataset, which allows to answer the question: "Is this food healthy or not?"

# Helper
https://stackoverflow.com/questions/61525486/bug-resolved-with-mount-drive-web-button-in-colab-accessing-shared-with-me

In [2]:
url = 'https://drive.google.com/open?id=1L4tDDnZWwPDjeRKlleoYHgic2_xNhZ8i'
file_id = '1L4tDDnZWwPDjeRKlleoYHgic2_xNhZ8i'
downloaded = drive.CreateFile({'id': file_id}) 
print(downloaded['title']) 
downloaded.GetContentFile('fr.openfoodfacts.org.products.csv')  
df = pd.read_csv('fr.openfoodfacts.org.products.csv', sep='\t') 

fr.openfoodfacts.org.products.csv


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,3,5,19,20,24,25,26,27,28,35,36,37,38,39,48) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(1)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_fr,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_fr,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_fr,ingredients_text,allergens,allergens_fr,traces,traces_tags,traces_fr,...,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,3087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Mission 1

Find a way to load this file into a pandas dataframe and print the first ten elements of the dataset.

Then print the number of columns which contain the types object and float.


In [4]:
df.head(10)

,code,url,creator,created_t,created_datetime,last_modified_t,last_modified_datetime,product_name,generic_name,quantity,packaging,packaging_tags,brands,brands_tags,categories,categories_tags,categories_fr,origins,origins_tags,manufacturing_places,manufacturing_places_tags,labels,labels_tags,labels_fr,emb_codes,emb_codes_tags,first_packaging_code_geo,cities,cities_tags,purchase_places,stores,countries,countries_tags,countries_fr,ingredients_text,allergens,allergens_fr,traces,traces_tags,traces_fr,...,vitamin-e_100g,vitamin-k_100g,vitamin-c_100g,vitamin-b1_100g,vitamin-b2_100g,vitamin-pp_100g,vitamin-b6_100g,vitamin-b9_100g,folates_100g,vitamin-b12_100g,biotin_100g,pantothenic-acid_100g,silica_100g,bicarbonate_100g,potassium_100g,chloride_100g,calcium_100g,phosphorus_100g,iron_100g,magnesium_100g,zinc_100g,copper_100g,manganese_100g,fluoride_100g,selenium_100g,chromium_100g,molybdenum_100g,iodine_100g,caffeine_100g,taurine_100g,ph_100g,fruits-vegetables-nuts_100g,collagen-meat-protein-ratio_100g,cocoa_100g,chlorophyl_100g,carbon-footprint_100g,nutrition-score-fr_100g,nutrition-score-uk_100g,glycemic-index_100g,water-hardness_100g
0,3087,http://world-fr.openfoodfacts.org/produit/0000...,openfoodfacts-contributors,1474103866,2016-09-17T09:17:46Z,1474103893,2016-09-17T09:18:13Z,Farine de blé noir,NaN,1kg,NaN,NaN,Ferme t'y R'nao,ferme-t-y-r-nao,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,en:FR,en:france,France,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4530,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Banana Chips Sweetened (Whole),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,États-Unis,"Bananas, vegetable oil (coconut oil, corn oil ...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0214,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,0.00129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.0,14.0,NaN,NaN
2,4559,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489069957,2017-03-09T14:32:37Z,1489069957,2017-03-09T14:32:37Z,Peanuts,NaN,NaN,NaN,NaN,Torn & Glasser,torn-glasser,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,États-Unis,"Peanuts, wheat flour, sugar, rice flour, tapio...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.071,NaN,0.00129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN
3,16087,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055731,2017-03-09T10:35:31Z,1489055731,2017-03-09T10:35:31Z,Organic Salted Nut Mix,NaN,NaN,NaN,NaN,Grizzlies,grizzlies,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,États-Unis,"Organic hazelnuts, organic cashews, organic wa...",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.143,NaN,0.00514,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.0,12.0,NaN,NaN
4,16094,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055653,2017-03-09T10:34:13Z,1489055653,2017-03-09T10:34:13Z,Organic Polenta,NaN,NaN,NaN,NaN,Bob's Red Mill,bob-s-red-mill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,US,en:united-states,États-Unis,Organic polenta,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,16100,http://world-fr.openfoodfacts.org/produit/0000...,usda-ndb-import,1489055651,2017-03-09T10:34:11Z,1489055651,2017-03-09T10:34:11Z,Breadshop Honey Gone Nuts Granola,NaN,NaN,NaN,NaN,Unfi,unfi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [5]:
print('cols with float or object:')
df.select_dtypes(include=['float64', 'object']).shape[1]

cols with float or object:


162

# Mission 2

Find a way to print firstly **only** object columns, then **only** float columns.

In [6]:
print(df.select_dtypes(include=['object']).head(1))
print(df.select_dtypes(include=['float64']).head(1))

   code  ... image_small_url
0  3087  ...             NaN

[1 rows x 56 columns]
   no_nutriments  additives_n  ...  glycemic-index_100g  water-hardness_100g
0            NaN          NaN  ...                  NaN                  NaN

[1 rows x 106 columns]


# Mission 3

Create a function that takes a dataframe as an argument and print the filling rate (the percentage of non-empty rows) of each of its columns. 

For example :

*   column 1 : 73%
*   column 2 : 99%


In [7]:
def get_fillrate(df):
  cols = df.columns
  for name in cols:
    percent = round(((df.shape[0] - df[name].isna().sum()) / (df.shape[0]))*100, 3)
    print(f"column '{name}' is {percent}% full")

In [8]:
#test function on sample
df_test = df.iloc[:, :5]
get_fillrate(df_test)
df['url'].isnull().sum(), df.shape[0]

column 'code' is 99.993% full
column 'url' is 99.993% full
column 'creator' is 99.999% full
column 'created_t' is 99.999% full
column 'created_datetime' is 99.997% full


(23, 320772)

# Mission 4

How many columns belong to each quartile of the filling rate? In other words, how many columns are filled at 0%, between 0% and 25%, between 25% and 50%...


In [9]:
def get_fillrate_quantile_counts(df):
  cols = df.columns
  q1, q2, q3, q4 = 0, 0, 0, 0
  for name in cols:
    percent = round(((df.shape[0] - df[name].isna().sum()) / (df.shape[0]))*100, 3)
    if percent <= 25:
      q1 +=1
    elif percent <= 50:
      q2 +=1
    elif percent <= 75:
      q3 +=1
    else:
      q4 +=1
  print(f"columns in q1 = {q1}, columns in q2 = {q2}, columns in q3 = {q3}, columns in q4 = {q4}")

In [10]:
get_fillrate_quantile_counts(df)

columns in q1 = 112, columns in q2 = 16, columns in q3 = 6, columns in q4 = 28


# Mission 5

We want to focus on the dishes available in France (since we want to launch our app there). 

To do this, select only the rows, where the value of 'countries_fr' contains the word "France".


In [11]:
mask = (df["countries_fr"].str.contains('France', na=False))
#df_France = df[mask]
df = df[mask]
#df_France["countries_fr"].value_counts()
df["countries_fr"].value_counts()


France                                                                     94392
France,Suisse                                                               1224
France,Royaume-Uni                                                           514
Belgique,France                                                              512
France,Allemagne                                                             240
                                                                           ...  
France,Italie,Monaco                                                           1
Belgique,France,Italie,Espagne                                                 1
Belgique,France,Allemagne,Irlande,Pays-Bas,Portugal,Espagne,Royaume-Uni        1
Belgique,France,Suisse,xx:Danemark                                             1
France,Slovénie                                                                1
Name: countries_fr, Length: 305, dtype: int64

# Mission 6

A dataset with too few data is not very interesting. After some analysis we realized that we can delete columns with more than 60% null values, except the columns pnns_groups_1, pnns_groups_2, nutrition-score-fr_100g and nutrition-score-uk_100g. Indeed these columns are going to be useful later.

If it's not done already, delete also the following columns since they won't help to solve our problem:
* creator
* created_t
* created_datetime
* last_modified_t
* brands_tags
* countries
* countries_tags
* additives_tags
* states
* states_tags

In [12]:
threshold = int(df.shape[0]*.6) #greater than 60% of values
exceptions = ["pnns_groups_1", "pnns_groups_2", "nutrition-score-fr_100g", "nutrition-score-uk_100g"]
to_drop = ["creator","created_t","created_datetime","last_modified_t","brands_tags",
"countries","countries_tags","additives_tags","states","states_tags"]
df.drop(to_drop, axis=1, inplace=True)
all_cols =  df.columns
cols_for_dropping = list(set(all_cols)-set(exceptions))
df = pd.concat([df.drop(exceptions, axis=1).dropna(axis=1, thresh=threshold, inplace=False), df[exceptions]], axis=1)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 98440 entries, 0 to 320770
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   code                     98440 non-null  object 
 1   url                      98440 non-null  object 
 2   last_modified_datetime   98440 non-null  object 
 3   product_name             91247 non-null  object 
 4   quantity                 71237 non-null  object 
 5   brands                   86426 non-null  object 
 6   categories               61955 non-null  object 
 7   categories_tags          61955 non-null  object 
 8   categories_fr            61955 non-null  object 
 9   countries_fr             98440 non-null  object 
 10  nutrition_grade_fr       61415 non-null  object 
 11  states_fr                98440 non-null  object 
 12  main_category            61955 non-null  object 
 13  main_category_fr         61955 non-null  object 
 14  energy_100g          

# Mission 7

Export you work in the csv format.


In [20]:
df.to_csv('france_food.csv')

In [21]:
!ls

adc.json  france_food.csv  fr.openfoodfacts.org.products.csv  sample_data
